In [1]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
from src.utils import etl
import importlib
importlib.reload(etl)
scraper = etl.ReviewScraper()

In [3]:
def get_driver(debug=False):
    options = Options()

    if not debug:
        options.add_argument("--headless")
    else:
        options.add_argument("--window-size=1366,768")

    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    driv_path = '../config/webdrivers/chromedriver.exe'
    input_driver = webdriver.Chrome(options=options, executable_path=driv_path)
    return input_driver

In [40]:
map_url

'https://maps.google.com/maps?q=Pupuseria+Salvadore%C3%B1a+1824+Westchester+Avenue,+The+Bronx&um=1&ie=UTF-8&sa=X&ved=2ahUKEwj8sdH-jNrvAhVFbc0KHVl7BfMQ_AUoAXoECAEQAw'

In [41]:
dpath = '../data/outputs/'+'cbg_restaurants/'

file_list = [dpath+ f for f in os.listdir(dpath)]
for file in file_list[1021:]:
    df = pd.read_json(file)
    for idx,i in df.iterrows():
        cbg, rest_name,rest_add,place_id = i['CBG'],i['name'],i['vicinity'],i['reference']
        rev_dpath = '../data/outputs/reviews/'
        
        fpath = rev_dpath + str(cbg)+ '_' + place_id + '.json'
        rev_files = [rev_dpath+ f for f in os.listdir(rev_dpath)]
        if 'Hotel' in rest_name or 'InterContinental' in rest_name or 'Plaza' in rest_name \
                or 'Inn' in rest_name or 'Leku' in rest_name or 'American Girl' in rest_name or 'Hilton' in rest_name:
            continue
        
        if fpath not in rev_files:
            url = 'https://www.google.com/search?q=' + rest_name + ' ' + rest_add
            driver = get_driver()
            driver.get(url)
            response = BeautifulSoup(driver.page_source, 'html.parser')

            for res in response.find_all('a', href=True):
                if 'maps.google' in res['href']:
                    map_url  = res['href']
            place_data = scraper.get_place_data(map_url)
            n_reviews = place_data['n_reviews']

            if n_reviews >0:
                if n_reviews > 500:
                    n_reviews = 0.8 * n_reviews
                revs = scraper.get_reviews(n_reviews,map_url,cbg,place_id)

    

WebDriverException: Message: unknown error: net::ERR_CONNECTION_CLOSED
  (Session info: headless chrome=89.0.4389.90)


In [487]:
df_test = pd.read_json('../data/outputs/reviews/10890021003_ChIJNRYaQ5lsYogRKbgbcK2R52w.json')
df_test

,id_review,caption,relative_date,retrieval_date,rating,username,n_review_user,n_photo_user,url_user,census_block_group
0,ChZDSUhNMG9nS0VJQ0FnSUR5NzhqZEdREAE,"GREAT PLACE TO EAT. FRIENDLY SERVICE, AND DELI...",15 hours ago,2021-03-23,5,Derek Hill,5,0,https://www.google.com/maps/contrib/1084165022...,10890021003
1,ChdDSUhNMG9nS0VJQ0FnSUR5cThpaHBBRRAB,,5 days ago,2021-03-23,5,LEE CHAMBERS,26,0,https://www.google.com/maps/contrib/1089573405...,10890021003
2,ChdDSUhNMG9nS0VJQ0FnSUR5aThPMXJBRRAB,Ok,5 days ago,2021-03-23,5,Alethia Lacy,0,0,https://www.google.com/maps/contrib/1084799071...,10890021003
3,ChZDSUhNMG9nS0VJQ0FnSUR5emZtemV3EAE,Love everything about it,a week ago,2021-03-23,5,Lynda Hall,103,0,https://www.google.com/maps/contrib/1132653948...,10890021003
4,ChZDSUhNMG9nS0VJQ0FnSUR5eGJXRkp3EAE,Best burger and fries I have ever tasted. Is l...,a week ago,2021-03-23,5,Mar Tina,0,0,https://www.google.com/maps/contrib/1059556461...,10890021003
...,...,...,...,...,...,...,...,...,...,...
925,ChZDSUhNMG9nS0VJQ0FnSURBc01idkNREAE,,4 years ago,2021-03-23,5,Royce Bryant,0,0,https://www.google.com/maps/contrib/1057974504...,10890021003
926,ChdDSUhNMG9nS0VJQ0FnSUN3em9IWnJRRRAB,So good. All of the seating is at the counter....,4 years ago,2021-03-23,5,Samuel Cordner,0,0,https://www.google.com/maps/contrib/1001352608...,10890021003
927,ChZDSUhNMG9nS0VJQ0FnSUNnNzl1MU5BEAE,,4 years ago,2021-03-23,5,Teresa Moore,5,0,https://www.google.com/maps/contrib/1183064930...,10890021003
928,ChZDSUhNMG9nS0VJQ0FnSUNBMnRhbFFREAE,"4 stars for fun and friendly service, 3 stars ...",4 years ago,2021-03-23,3,Beau Helton,0,0,https://www.google.com/maps/contrib/1177112957...,10890021003


In [ ]:
df_rev['rev_length'] = df_rev['caption'].str.len()
df_rev[['rating', 'rev_length']].corr()